# PDF yüklemesinin gerçekleştirilmesi

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("Anayasa.pdf")
data = loader.load()

In [2]:
len(data)

74

# Veriyi parçalara ayırma (Chunking İşlemi)

Tek bir büyük metin parçası yerine, dokümantasyonu daha küçük parçalara böldük? Doğru bilgiyi daha kolay bulmasını sağlayabilmek için bu işlemi gerçekleştirdik.
Metni 1000 karakterlik parçalar halinde bölerek işlemeye hazır hale getiriyoruz.

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000)
docs = text_splitter.split_documents(data)

print("Total", len(docs))

Total 272


In [5]:
docs[7]

Document(metadata={'producer': 'Microsoft® Word 2013', 'creator': 'Microsoft® Word 2013', 'creationdate': '2020-03-04T17:29:51+03:00', 'author': 'Canan Baysal', 'moddate': '2020-03-04T17:29:51+03:00', 'source': 'Anayasa.pdf', 'total_pages': 74, 'page': 2, 'page_label': '3'}, page_content='3 \n \nMadde 5 – Devletin temel amaç ve görevleri, Türk milletinin bağımsızlığını ve bütünlüğünü, \nülkenin bölünmezliğini, Cumhuriyeti ve demo krasiyi korumak, kişilerin ve toplumun refah, huzur ve \nmutluluğunu sağlamak; kişinin temel hak ve hürriyetlerini, sosyal hukuk devleti ve adalet ilkeleriyle \nbağdaşmayacak surette sınırlayan siyasal, ekonomik ve sosyal engelleri kaldırmaya, insanın maddi ve  \nmanevi varlığının gelişmesi için gerekli şartları hazırlamaya çalışmaktır. \n \nVI. Egemenlik \nMadde 6 – Egemenlik, kayıtsız şartsız Milletindir. \nTürk Milleti, egemenliğini, Anayasanın koyduğu esaslara göre, yetkili organları eliyle kullanır. \nEgemenliğin kullanılması, hiçbir surette hiçbir kişiye

# Google Generative AI Embeddings'i kullanarak Embedding oluşturma kısmı

Text veriyi sayısal bir veriye çevirmeye çalıştık.

In [8]:
from langchain_chroma import Chroma
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from dotenv import load_dotenv

load_dotenv()

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
vector = embeddings.embed_query("hello")
vector[:5]

[0.017157701775431633,
 -0.009304771199822426,
 -0.011424741707742214,
 -0.045027956366539,
 0.029878143221139908]

# ChromaDB üzerine kayıtı işlemi

In [9]:
vectorstore = Chroma.from_documents(documents=docs,embedding=embeddings)

In [10]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs= {"k" : 10})  

In [11]:
retrieved_docs = retriever.invoke("Devletin şekli nedir?")

In [12]:
len(retrieved_docs)

10

In [14]:
#print(retrieved_docs[5].page_content)

# Google Gemini API yapısını kullanarak LLM Invoke işlemleri

In [ ]:
#Temperature Değer Aralığı

#Düşük Değerler(0.1-0.4):Daha kesin ve daha tutarlı cevaplar vermemizi sağlayabilir. Model daha tahmin edilebilir hale gelir.
#Orta Değerler(0.4-0.7): Hem mantıklı hem de yaratıcı cevaplar verebiliyor.\n
#Yüksek Değerler(0.8-1.0) : Daha rastgele ve yaratıcı cevaplar ve bazen de belki tutarsız cevaplar verebilir.

In [15]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0.3,
    max_tokens=500
)

In [16]:
from langchain.chains import create_retrieval_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

In [17]:
system_prompt = (
    "Sen bir yardımcı asistansın ve yalnızca T.C. anayasası ve anayasa maddeleri hakkında sorulara cevap veriyorsun. "
    "Yanıtlarını yalnızca verilen bağlam içeriğinden oluştur. "
    "Eğer sorunun cevabını bilmiyorsan, 'Bu konuda yardımcı olamıyorum.' de. "
    "Cevaplarını en fazla üç cümle ile ver ve doğru bilgi içerdiğinden emin ol.\n\n"
    "{context}"
)

In [18]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

# Soru-Cevap Üretme Zinciri Oluşturu (LLM+PROMPT)

In [19]:
question_answer_chain = create_stuff_documents_chain(llm,prompt)

# RAG Zinciri Oluşturma ( RAG + LLM Entegrasyonun Gerçekleşmesi )

In [20]:
rag_chain = create_retrieval_chain(retriever,question_answer_chain)

In [21]:
# Kullanıcı Sorgusunu Çalıştırarak Cevap Üretme İşlemi

In [22]:
response = rag_chain.invoke({"input":"Devletin şekli nedir?"})

In [25]:
#print(response)

In [24]:
print(response["answer"]);

Türkiye Devleti bir Cumhuriyettir.
